<a href="https://colab.research.google.com/github/EmanuelPutura/Text-to-SQL/blob/main/docs/google_t5_finetuned_text_to_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring the Potential of Using Transfer Learning with Google T5 Model for Text-to-SQL Tasks

Author: **Emanuel-Vasile Putura** - Department of Computer Science, Babes-Bolyai University


> The code is inspired from a number of different sources, the main one being this [Google Colab](https://github.com/patil-suraj/exploring-T5/blob/master/t5_fine_tuning.ipynb) written by Suraj Patil.

> This is the [Github repository](https://github.com/EmanuelPutura/Text-to-SQL) for this project. There you can find the documentation for it, a presentation poster and a short teaser video.

We use the Google T5 model and apply transfer learning to it for our text-to-SQL task. The dataset we are using is the [WikiSQL dataset](https://github.com/salesforce/WikiSQL).

In [4]:
! pip install -q transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.4 MB/s eta 0:00:00


In [5]:
! pip install -q rouge_score

  Preparing metadata (setup.py) ... done


In [6]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
    
Token: 
Add token as git credential? (Y/n) Y
Token is valid.
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper

In [8]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

T5_MODEL = 't5-small'

tokenizer = AutoTokenizer.from_pretrained(T5_MODEL)
model = T5ForConditionalGeneration.from_pretrained(T5_MODEL)

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [9]:
from datasets import load_dataset

train_data = load_dataset('wikisql', split='train+validation')
test_data = load_dataset('wikisql', split='test')

Generating test split:   0%|          | 0/15878 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8421 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/56355 [00:00<?, ? examples/s]

Dataset wikisql downloaded and prepared to /root/.cache/huggingface/datasets/wikisql/default/0.1.0/7037bfe6a42b1ca2b6ac3ccacba5253b1825d31379e9cc626fc79a620977252d. Subsequent calls will reuse this data.


This is the current format of the dataset, which we have to modify to be able to use it with the Google T5 model.

In [10]:
train_data

Dataset({
    features: ['phase', 'question', 'table', 'sql'],
    num_rows: 64776
})

In [11]:
def format_dataset(example):
  return {'input': 'translate to SQL: ' + example['question'], 'target': example['sql']['human_readable']}

In [12]:
train_data = train_data.map(format_dataset, remove_columns=train_data.column_names)

  0%|          | 0/64776 [00:00<?, ?ex/s]

In [13]:
test_data = test_data.map(format_dataset, remove_columns=test_data.column_names)

  0%|          | 0/15878 [00:00<?, ?ex/s]

This is the enew format of the dataset, after preprocessing it.

In [14]:
train_data

Dataset({
    features: ['input', 'target'],
    num_rows: 64776
})

### Let's calculate the inputs and targets lengths

In [15]:
# map article and summary len to dict as well as if sample is longer than 512 tokens
def map_to_length(x):
  x["input_len"] = len(tokenizer(x["input"]).input_ids)
  x["input_longer_256"] = int(x["input_len"] > 256)
  x["input_longer_128"] = int(x["input_len"] > 128)
  x["input_longer_64"] = int(x["input_len"] > 64)
  x["out_len"] = len(tokenizer(x["target"]).input_ids)
  x["out_longer_256"] = int(x["out_len"] > 256)
  x["out_longer_128"] = int(x["out_len"] > 128)
  x["out_longer_64"] = int(x["out_len"] > 64)
  return x

sample_size = 10000
data_stats = train_data.select(range(sample_size)).map(map_to_length, num_proc=4)

def compute_and_print_stats(x):
  if len(x["input_len"]) == sample_size:
    print(
        "Input Mean: {}, %-Input > 256:{},  %-Input > 128:{}, %-Input > 64:{} Output Mean:{}, %-Output > 256:{}, %-Output > 128:{}, %-Output > 64:{}".format(
            sum(x["input_len"]) / sample_size,
            sum(x["input_longer_256"]) / sample_size,
            sum(x["input_longer_128"]) / sample_size,
            sum(x["input_longer_64"]) / sample_size,   
            sum(x["out_len"]) / sample_size,
            sum(x["out_longer_256"]) / sample_size,
            sum(x["out_longer_128"]) / sample_size,
            sum(x["out_longer_64"]) / sample_size,
        )
    )

output = data_stats.map(
  compute_and_print_stats, 
  batched=True,
  batch_size=-1,
)

#1:   0%|          | 0/2500 [00:00<?, ?ex/s]

#2:   0%|          | 0/2500 [00:00<?, ?ex/s]

#3:   0%|          | 0/2500 [00:00<?, ?ex/s]

#0:   0%|          | 0/2500 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Input Mean: 19.8508, %-Input > 256:0.0,  %-Input > 128:0.0, %-Input > 64:0.0002 Output Mean:20.0403, %-Output > 256:0.0, %-Output > 128:0.0002, %-Output > 64:0.0005


Here we are going to tokenize our data.

In [16]:
# tokenize the examples
def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['input'], pad_to_max_length=True, max_length=64)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target'], pad_to_max_length=True, max_length=64)

    encodings = {
        'input_ids': input_encodings['input_ids'], 
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids'],
        'decoder_attention_mask': target_encodings['attention_mask']
    }

    return encodings

In [17]:
train_data = train_data.map(convert_to_features, batched=True, remove_columns=train_data.column_names)
test_data = test_data.map(convert_to_features, batched=True, remove_columns=test_data.column_names)

columns = ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask']

train_data.set_format(type='torch', columns=columns)
test_data.set_format(type='torch', columns=columns)

  0%|          | 0/65 [00:00<?, ?ba/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


  0%|          | 0/16 [00:00<?, ?ba/s]

In [18]:
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments
from transformers import get_linear_schedule_with_warmup

## Preparing the training phase

Choose the training arguments you want to use for training the model.

In [20]:
PATH_TO_TRAINED_MODEL = '/content/pretrained'

In [21]:
# Option 1
#   Batch size: 16
#   Epochs number: 5

# set training arguments - Feel free to adapt it
training_args = Seq2SeqTrainingArguments(
    output_dir=PATH_TO_TRAINED_MODEL,
    per_device_train_batch_size=16,
    num_train_epochs=5,
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    evaluation_strategy="epoch",
    do_train=True,
    do_eval=True,
    logging_steps=500,
    save_strategy="epoch",
    overwrite_output_dir=True,
    save_total_limit=3,
    load_best_model_at_end=True
)

In [15]:
# Option 2
#   Batch size: 32
#   Epochs number: 10

# set training arguments - Feel free to adapt it
training_args = Seq2SeqTrainingArguments(
    output_dir=PATH_TO_TRAINED_MODEL,
    per_device_train_batch_size=32,
    num_train_epochs=5,
    per_device_eval_batch_size=32,
    predict_with_generate=True,
    evaluation_strategy="steps",
    eval_steps = 1000,
    do_train=True,
    do_eval=True,
    logging_steps=500,
    save_strategy="steps",
    save_steps= 1000,
    overwrite_output_dir=True,
    save_total_limit=3,
    load_best_model_at_end=True
)

## Rouge metrics for evaluating our model

In [22]:
from datasets import load_metric
rouge = load_metric("rouge")

def compute_metrics(pred):
  labels_ids = pred.label_ids
  pred_ids = pred.predictions

  # all unnecessary tokens are removed
  pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
  labels_ids[labels_ids == -100] = tokenizer.pad_token_id
  label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

  rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

  return {
      "rouge2_precision": round(rouge_output.precision, 4),
      "rouge2_recall": round(rouge_output.recall, 4),
      "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
  }

<ipython-input-22-ff41f60595c8>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


## Create the Trainer instance
> Feel free to choose an option, depending on the optimizer

In [23]:
# Option 1: Default Optimizer (Adam)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=test_data,
)

In [29]:
# Option 2: AdamW Optimizer

from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_data) * 20)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=test_data,
    optimizers=(optimizer, scheduler)
)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/content/t5-small-finetuned-wikisql is already a clone of https://huggingface.co/emanuelputura/t5-small-finetuned-wikisql. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 15878
  Batch size = 16


## Train our model

In [31]:
trainer.train()

***** Running training *****
  Num examples = 64776
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 20245
  Number of trainable parameters = 60506624


Epoch,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 15878
  Batch size = 16


Epoch,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
1,0.203800,0.165803,0.786100,0.697100,0.731800
2,0.179900,0.149516,0.800100,0.708600,0.744500
3,0.167200,0.140085,0.806600,0.715400,0.751300
4,0.158700,0.133304,0.811300,0.719600,0.755700
5,0.149600,0.128648,0.814500,0.722500,0.758800


Saving model checkpoint to /content/t5-small-finetuned-wikisql/checkpoint-4049
Configuration saved in /content/t5-small-finetuned-wikisql/checkpoint-4049/config.json
Model weights saved in /content/t5-small-finetuned-wikisql/checkpoint-4049/pytorch_model.bin
Several commits (2) will be pushed upstream.
***** Running Evaluation *****
  Num examples = 15878
  Batch size = 16
Saving model checkpoint to /content/t5-small-finetuned-wikisql/checkpoint-8098
Configuration saved in /content/t5-small-finetuned-wikisql/checkpoint-8098/config.json
Model weights saved in /content/t5-small-finetuned-wikisql/checkpoint-8098/pytorch_model.bin
Deleting older checkpoint [/content/t5-small-finetuned-wikisql/checkpoint-1000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 15878
  Batch size = 16
Saving model checkpoint to /content/t5-small-finetuned-wikisql/checkpoint-12147
Configuration saved in /content/t5-small-finetuned-wikisql/checkpoint-12147/config.json
Model weights sa

TrainOutput(global_step=20245, training_loss=0.17725811706527894, metrics={'train_runtime': 4348.3303, 'train_samples_per_second': 74.484, 'train_steps_per_second': 4.656, 'total_flos': 5479312832593920.0, 'train_loss': 0.17725811706527894, 'epoch': 5.0})

In [32]:
trainer.save_model()

Saving model checkpoint to /content/t5-small-finetuned-wikisql
Configuration saved in /content/t5-small-finetuned-wikisql/config.json
Model weights saved in /content/t5-small-finetuned-wikisql/pytorch_model.bin
Saving model checkpoint to /content/t5-small-finetuned-wikisql
Configuration saved in /content/t5-small-finetuned-wikisql/config.json
Model weights saved in /content/t5-small-finetuned-wikisql/pytorch_model.bin
Several commits (3) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.30k/231M [00:00<?, ?B/s]

Upload file runs/Jan15_11-48-56_1d9c08365794/events.out.tfevents.1673783601.1d9c08365794.569.2:  24%|##4      …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/emanuelputura/t5-small-finetuned-wikisql
   6bb3255..1fe1289  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/emanuelputura/t5-small-finetuned-wikisql
   6bb3255..1fe1289  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}, 'dataset': {'name': 'wikisql', 'type': 'wikisql', 'config': 'default', 'split': 'train+validation', 'args': 'default'}}
To https://huggingface.co/emanuelputura/t5-small-finetuned-wikisql
   1fe1289..ec39758  main -> main

   1fe1289..ec39758  main -> main



In [33]:
tokenizer.save_pretrained(PATH_TO_TRAINED_MODEL)

tokenizer config file saved in /content/t5-small-finetuned-wikisql/tokenizer_config.json
Special tokens file saved in /content/t5-small-finetuned-wikisql/special_tokens_map.json


('/content/t5-small-finetuned-wikisql/tokenizer_config.json',
 '/content/t5-small-finetuned-wikisql/special_tokens_map.json',
 '/content/t5-small-finetuned-wikisql/tokenizer.json')

In [34]:
trainer.create_model_card()

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}, 'dataset': {'name': 'wikisql', 'type': 'wikisql', 'config': 'default', 'split': 'train+validation', 'args': 'default'}}


## Test the model

In [36]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained(PATH_TO_TRAINED_MODEL)
model = T5ForConditionalGeneration.from_pretrained(PATH_TO_TRAINED_MODEL)

loading file spiece.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file /content/t5-small-finetuned-wikisql/config.json
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_leng

In [37]:
test_data = load_dataset('wikisql', split='test')

In [38]:
def translate_to_sql(text):
  inputs = tokenizer(text, padding='longest', max_length=64, return_tensors='pt')
  input_ids = inputs.input_ids
  attention_mask = inputs.attention_mask
  output = model.generate(input_ids, attention_mask=attention_mask, max_length=64)

  return tokenizer.decode(output[0], skip_special_tokens=True)

In [39]:
for i in range(0,100,10):
  print('Natural language query: ' + test_data[i]['question'])
  print('Predicted query: ' + translate_to_sql('translate to SQL: ' + test_data[i]['question']))
  print('Actual query: ' + test_data[i]['sql']['human_readable'])
  print('=================================\n')

translate to SQL: What is terrence ross' nationality
Predict. :SELECT Nationality FROM table WHERE Name = terrence ross
Expected: SELECT Nationality FROM table WHERE Player = Terrence Ross

translate to SQL: How many different nationalities do the players of New Jersey Devils come from?
Predict. :SELECT COUNT Nationality FROM table WHERE Team = New Jersey Devils
Expected: SELECT COUNT Nationality FROM table WHERE NHL team = New Jersey Devils

translate to SQL: If you are a pilot officer in the commonwealth then what will you called as in the US air force?
Predict. :SELECT US air force FROM table WHERE Position = pilot officer AND Country = commonwealth
Expected: SELECT US Air Force equivalent FROM table WHERE Commonwealth equivalent = Pilot Officer

translate to SQL: what is the total number of total w–l where doubles w–l is 11–11
Predict. :SELECT COUNT Total W–L FROM table WHERE Doubles W–L = 11–11
Expected: SELECT COUNT Total W–L FROM table WHERE Doubles W–L = 11–11

translate to SQL